# Demo Notebook for using the Reboot Motion Recommendation Engine

__[CoLab Notebook Link](https://githubtocolab.com/RebootMotion/reboot-toolkit/blob/main/examples/Recommendations.ipynb)__

Run the cells in order, making sure to enter AWS credentials in the cell when prompted

In [ ]:
#@title Install Python Packages

!pip install git+https://github.com/RebootMotion/reboot-toolkit.git@v2.7.1#egg=reboot_toolkit > /dev/null
!pip install git+https://github.com/RebootMotion/mlb-statsapi.git@v1.1.0#egg=mlb_statsapi > /dev/null
!echo "Done Installing"

In [ ]:
#@title Import Statements

import reboot_toolkit as rtk
from reboot_toolkit import MovementType, setup_aws, MovementType, MocapType, Handedness, FileType, S3Metadata, PlayerMetadata, decorate_primary_segment_df_with_stats_api
import os
import pandas as pd

In [ ]:
#@title AWS Credentials

# Upload your Organization's .env file to the local file system, per https://pypi.org/project/python-dotenv/
# OR input your credentials string generated by the Reboot Dashboard

boto3_session = setup_aws()

In [ ]:
#@title User Input - No code changes required below this section, just enter information in forms

# Update the below info to match your desired analysis information
# Common changes you might want to make:

# To analyze both Hawk-Eye HFR data from the Stats API,
# and also Hawk-Eye Action files (e.g. from the DSP),
#  set mocap_types=[MocapType.HAWKEYE_HFR, MocapType.HAWKEYE]

# To analyze baseball-hitting,
# set movement_type=MovementType.BASEBALL_HITTING

# To analyze right-handed players,
# set handedness=Handedness.RIGHT

# To analyze data from the momentum and energy files,
# set file_type=FileType.MOMENTUM_ENERGY

# See https://docs.rebootmotion.com/ for all available file types and the data in each
mocap_types = [MocapType.HAWKEYE, MocapType.HAWKEYE_HFR]
movement_type = MovementType.BASEBALL_PITCHING
handedness = Handedness.LEFT
file_type = FileType.METRICS_BASEBALL_PITCHING_V_ALL

# Update the label to whatever you'd like to be displayed in the visuals
primary_segment_label = 'Primary Segment'

# Use this bool to add columns of data, like pitch_type and start_speed, from the stats API
add_stats_api = True  # True or False

if add_stats_api:
    print("Will add data from the Stats API like velo and pitch type")
    
else:
    print("Will NOT add data from the Stats API like velo and pitch type (set to True above if needed)")

In [ ]:
#@title Set S3 File Info

s3_metadata = S3Metadata(
    org_id=os.environ['ORG_ID'],
    mocap_types=mocap_types,
    movement_type=movement_type,
    handedness=handedness,
    file_type=file_type,
)

s3_df = rtk.download_s3_summary_df(s3_metadata)

In [ ]:
#@title Optional Look Up Player by Name

name_to_look_up = "Jacob deGrom"

rtk.find_player_matches(s3_df, name_to_look_up, match_threshold=50., max_results=5)

In [ ]:
#@title Display the Interface for Selecting the Primary Data Segment to Analyze

primary_segment_widget = rtk.create_interactive_widget(s3_df)
display(primary_segment_widget)

In [ ]:
#@title Set Primary Analysis Segment Info

primary_segment_data = primary_segment_widget.children[1].result
primary_analysis_segment = PlayerMetadata(
    org_player_ids=primary_segment_data["org_player_ids"],
    session_dates=primary_segment_data["session_dates"],
    session_nums=primary_segment_data["session_nums"],
    session_date_start=primary_segment_data["session_date_start"],
    session_date_end=primary_segment_data["session_date_end"],
    year=primary_segment_data["year"],
    org_movement_id=None, # set the play GUID for the skeleton animation; None defaults to the first play
    s3_metadata=s3_metadata,
)

primary_segment_summary_df = rtk.filter_s3_summary_df(primary_analysis_segment, s3_df)

# Add Movement Num and S3 Key to Primary DataFrame to Enable Sorting
prim_available_s3_keys = rtk.list_available_s3_keys(os.environ['ORG_ID'], primary_segment_summary_df)
primary_segment_data_df = rtk.load_games_to_df_from_s3_paths(primary_segment_summary_df['s3_path_delivery'].tolist())
primary_segment_data_df = rtk.merge_data_df_with_s3_keys(primary_segment_data_df, prim_available_s3_keys)

if add_stats_api:
    print('Adding Stats API data (like pitch speed) to the data df...')
    primary_segment_data_df = decorate_primary_segment_df_with_stats_api(primary_segment_data_df)
    print("Available Pitch Types:")
    print(primary_segment_data_df['pitch_type'].unique())

In [ ]:
#@title Optional: After adding the Stats API data, uncomment below to filter the data

# # FILTER BY PITCH TYPES
# pitch_types = {'Four-Seam Fastball', 'Curveball'}  # list the pitch types you want to include
# primary_segment_data_df = primary_segment_data_df.loc[
#     primary_segment_data_df['pitch_type'].isin(pitch_types)
# ].copy().reset_index(drop=True)

# # FILTER BY A VELO RANGE
# velo_lo = 90
# velo_hi = 100
# primary_segment_data_df = primary_segment_data_df[
#     (primary_segment_data_df["start_speed"] > velo_lo) & (primary_segment_data_df["start_speed"] < velo_hi)
# ].copy().reset_index(drop=True)

# # Uncomment to print number of rows returned by filters
# print('Num available rows:', len(primary_segment_data_df))

In [ ]:
#@title Run metrics against Reboot Recommendation Engine

"""
This is a recommendation engine with the goal of giving coaches guidance on how to help their players improve.
This recommendation engine is trained on player metrics as a predictor of fastball velocity for pitching and bat velocity for hitting. 
Using a random forest like model for training, the recommendation engine then extracts the impact of individual metrics on the overall result.
Given a player's metrics, the engine is then able to recommend aspects to focus on to improve the overall result.
"""

num_features_to_plot = 5  # might be able to increase by 1 or 2 and still have a result returned, but more than that and the file gets too large

desired_mocap_type = None  # you can input mocap_types[0], or an input of None will infer the mocap type from the file 

df = rtk.recommendation(
    boto3_session, primary_segment_data_df, movement_type, mocap_types[0], handedness
)
fig = rtk.recommendation_violin_plot(
    boto3_session, primary_segment_data_df, movement_type, desired_mocap_type, handedness, 
    num_features=num_features_to_plot
)

fig.show()
df